In [28]:
with open('/home/jeongyeojin/dataset_286_4.txt') as f:
    data = f.read()
    dna_data = data.split('\n')[:-1]
dna_data

['(+83 -76 +128 -70 -74 +130 +46 -104 -121 -37 +10 -5 +124 -93 -82 -55 +127 -34 -32 +111 -25 -50 -22 +133 -6 -126 +75 +114 +54 +120 +42 -119 +91 -110 +2 +8 +71 +36 +57 -92 -40 +47 +107 +24 -96 +103 -132 -39 -87 -102 +106 +56 +64 +43 +73 -44 -20 +14 +112 +95 +88 -12 +118 +108 -21 -26 -31 -60 +122 -84 +48 +53 +13 +125 +72 -79 +61 +78 +65 -16 -109 +1 -7 -4 +68 +35 +17 -58 +131 -99 +81 +51 +115 +117 +66 -63 -129 -94 +62 -41 -29 +123 +33 -90 -69 +38 +67 +19 +52 -86 +89 -30 +15 -27 -101 -85 +23 -80 +3 +97 -9 +59 -49 -11 -18 +28 +113 +105 -98 +45 +100 +77 +116)']

In [26]:
# ex. P : '(-3 +1 +2)
import re

def greedy_sorting(P):
    print_list = lambda x : print("(" + " ".join(["+"+str(s) if s>0 else str(s) for s in x]) + ")")
    approx_reversal_distance = 0
    p_list = [int(s) for s in re.split('[( )]', P)[1:-1]] 
    for i in range(len(p_list)):
        # single flip
        if p_list[i] == -(i+1):
            p_list[i] *= -1
            approx_reversal_distance += 1
            print_list(p_list)
        elif p_list[i] != (i+1):
            if (i+1) in p_list[i+1:]:
                target_index = p_list.index(i+1)
                p_list = p_list[:i] + [-s for s in p_list[i:target_index+1][::-1]] + p_list[target_index+1:]
                approx_reversal_distance += 1
                print_list(p_list)
                p_list[i] *= -1
                approx_reversal_distance += 1
                print_list(p_list)
            elif -(i+1) in p_list[i+1:]:
                target_index = p_list.index(-(i+1))
                p_list = p_list[:i] + [-s for s in p_list[i:target_index+1][::-1]] + p_list[target_index+1:]
                approx_reversal_distance += 1
                print_list(p_list)
                